In [ ]:
import pandas as pd
import requests
import time
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# Kakao API 키 설정 (https://developers.kakao.com/ 에서 발급)
# 환경 변수에서 KAKAO_API_KEY 가져오기
KAKAO_API_KEY = os.getenv('KAKAO_API_KEY')

In [ ]:
data = 'data/20251116003832_외국인 관심 관광지_영어권.csv'

df = pd.read_csv(data)

# 괄호 안의 한글 데이터만 추출
df['관광지명_한글'] = df['관광지명'].str.extract(r'\(([^)]+)\)', expand=False)

def get_address_kakao(place_name):
    """Kakao Maps API로 장소명에 대한 주소 검색"""
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_API_KEY}'}
    params = {'query': place_name}

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                # 첫 번째 검색 결과 반환
                place = result['documents'][0]
                return {
                    'address': place.get('address_name', ''),
                    'road_address': place.get('road_address_name', ''),
                    'x': place.get('x', ''),  # 경도
                    'y': place.get('y', '')   # 위도
                }
        time.sleep(0.1)  # API 호출 제한 방지
    except Exception as e:
        print(f"Error for {place_name}: {e}")

    return {'address': '', 'road_address': '', 'x': '', 'y': ''}

# 주소 정보 가져오기
print("주소 정보를 가져오는 중...")
address_data = df['관광지명_한글'].apply(get_address_kakao)
df['지번주소'] = address_data.apply(lambda x: x['address'])
df['도로명주소'] = address_data.apply(lambda x: x['road_address'])
df['경도'] = address_data.apply(lambda x: x['x'])
df['위도'] = address_data.apply(lambda x: x['y'])

print(df[['관광지명_한글', '도로명주소', '지번주소']].head())

# 결과 저장
df.to_csv('data/tourist_spots.csv', index=False, encoding='utf-8')

In [13]:
df = pd.read_csv('../data/tourist_spots.csv')

df.head()

df[['관광지명_한글', '경도', '위도']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   관광지명_한글  99 non-null     object 
 1   경도       95 non-null     float64
 2   위도       95 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [14]:
# 위도 또는 경도가 비어있는 행 확인
missing_coords = df[(df['경도'].isna()) | (df['위도'].isna())]

print(f"좌표 정보가 없는 관광지 수: {len(missing_coords)}")
print("\n좌표 정보가 없는 관광지 목록:")
display(missing_coords[['관광지명_한글', '경도', '위도']])

# 좌표 정보 통계
print("=== 좌표 정보 통계 ===")
print(f"전체 관광지 수: {len(df)}")
print(f"좌표 있는 관광지: {df['경도'].notna().sum()}개")
print(f"좌표 없는 관광지: {df['경도'].isna().sum()}개")
print(f"좌표 완성도: {df['경도'].notna().sum() / len(df) * 100:.2f}%")

좌표 정보가 없는 관광지 수: 5

좌표 정보가 없는 관광지 목록:


,관광지명_한글,경도,위도
9,이태원 쇼핑거리,NaN,NaN
16,Hongik University Street,NaN,NaN
43,수문장 교대의식,NaN,NaN
61,덕수궁 왕궁수문장교대의식,NaN,NaN
91,인사동 고미술거리,NaN,NaN


=== 좌표 정보 통계 ===
전체 관광지 수: 100
좌표 있는 관광지: 95개
좌표 없는 관광지: 5개
좌표 완성도: 95.00%
